# # libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [2]:
address_jejudsi="https://jejudsi.kr/issue.htm"

In [3]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/chromedriver/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
d.get(address_jejudsi)
req = requests.get(address_jejudsi,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

# # 기타 프로젝트 수집

In [ ]:
# 가치더함 진입
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
address_jejudsi="https://jejudsi.kr/issue.htm"
d.get(address_jejudsi)
req = requests.get(address_jejudsi,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

# 기타 단추 클릭
xpath_click_etc = '/html/body/div[1]/section/div/div/div[2]/div/div/div/div/nav/div[2]/ul/li[9]/a'
etc = d.find_element(By.XPATH,xpath_click_etc).text
d.find_element(By.XPATH,xpath_click_etc).click()

# 더보기 무한 클릭
path_more ='/html/body/div[1]/section/div/div/div[2]/div/div/div/div/div[7]/div[2]/a'
while True :
    try :
        d.find_element(By.XPATH,path_more).click()
    except :
        break  

# 새로운 데이터프레임 생성
col = ['분류'
       ,'프로젝트 종류'
       ,'프로젝트 시작'
       ,'제목'
       ,'제안의시작'
       ,'주요제안(문제정의)'
       ,'해결방안'
       ,'공감투표 참여안내'
      ]
df = pd.DataFrame(columns=col) 
        
# 프로젝트 진입 주소 수집
html_2 = d.page_source
soup_2 = BeautifulSoup(html_2, "html.parser")
list_issueTit = soup_2.find_all('div', attrs={"class" : "issue-tit"})

list_tag_issueTit = list()

for i_list_issueTit in list_issueTit :
    list_tag_issueTit.append(i_list_issueTit.a['href'])
    
for i_list_tag_issueTit in list_tag_issueTit :
    # 프로젝트 진입
    d.get(f'https://jejudsi.kr{i_list_tag_issueTit}')
    
    # 프로젝트 종류
    path_status ='/html/body/div[1]/section/div[1]/div/p[1]/span'
    status = d.find_element(By.XPATH, path_status).text    
    
    # 프로젝트 시작 시점
    try :        
        path_start = '/html/body/div[1]/section/div[2]/div/div[3]/div[1]/div[1]/div/div[2]/h3'
        start = d.find_element(By.XPATH, path_start).text
        start = start[9:19]
    except :
        start = np.nan

    # 프로젝트 제목
    title = d.find_element(By.CLASS_NAME, 'text-center').text

    # 내용 수집

    html_test = d.page_source
    soup_test = BeautifulSoup(html_test, "html.parser")

    # mt-3 섹션
    list_test = soup_test.find_all('div', attrs={"class" : "card mt-3"})

    list_list_test = list()
    for i_list_test in list_test :
        cleantext = i_list_test.text
        i_list_test = re.sub("\n", ".", cleantext)
        list_list_test.append(i_list_test)
    list_list_test

    cleantext_reason = list()
    cleantext_content = list()
    cleantext_cuase = list()
    cleantext_suggestion = list()
    cleantext_solution = list()
    cleantext_vote = list()
    cleantext_reference = list()

    for i_list_list_test in list_list_test :        
        if '미추진 사유' in i_list_list_test[:15] :
            cleantext_reason.appen(i_list_list_test)
        elif '상세 내용' in i_list_list_test[:15] :
            cleantext_content.append(i_list_list_test)
        elif '제안의 시작' in i_list_list_test[:15] :
            cleantext_cuase.append(i_list_list_test) 
        elif '주요제안' in i_list_list_test[:15] :
            cleantext_suggestion.append(i_list_list_test)
        elif '해결방안' in i_list_list_test[:15] :
            cleantext_solution.append(i_list_list_test)          
        elif '공감투표' in i_list_list_test[:15] :
            cleantext_vote.append(i_list_list_test)
        elif '관련 자료' in i_list_list_test[:15] :
            cleantext_reference.append(i_list_list_test)
        elif '댓글' in i_list_list_test[:15] :
            pass
        else :
            pass

    if not (cleantext_reason and \
          cleantext_content and \
          cleantext_cuase and \
          cleantext_suggestion and \
          cleantext_solution and \
          cleantext_vote and \
          cleantext_reference) :

        # mt-t-10 섹션
        list_test = soup_test.find_all('div', attrs={"class" : "card m-t-10"})

        list_list_test = list()
        for i_list_test in list_test :
            cleantext = i_list_test.text
            i_list_test = re.sub("\n", ".", cleantext)
            list_list_test.append(i_list_test)

        for i_list_list_test in list_list_test :        
            if '미추진 사유' in i_list_list_test[:15] :
                cleantext_reason.append(i_list_list_test)
            elif '상세 내용' in i_list_list_test[:15] :
                cleantext_content.append(i_list_list_test)
            elif '제안의 시작' in i_list_list_test[:15] :
                cleantext_cuase.append(i_list_list_test) 
            elif '주요제안' in i_list_list_test[:15] :
                cleantext_suggestion.append(i_list_list_test)
            elif '해결방안' in i_list_list_test[:15] :
                cleantext_solution.append(i_list_list_test)          
            elif '공감투표' in i_list_list_test[:15] :
                cleantext_vote.append(i_list_list_test)
            elif '관련 자료' in i_list_list_test[:15] :
                cleantext_reference.append(i_list_list_test)
            elif '댓글' in i_list_list_test[:15] :
                pass
            else :
                pass
    else :
        pass
    
    if not cleantext_reason :
        cleantext_reason = np.nan
    else :
        cleantext_reason = cleantext_reason[0]
    if not cleantext_content :
        cleantext_content = np.nan
    else :
        cleantext_content = cleantext_content[0]
    if not cleantext_cuase :
        cleantext_cuase = np.nan
    else :
        cleantext_cuase = cleantext_cuase[0]
    if not cleantext_suggestion :
        cleantext_suggestion = np.nan
    else :
        cleantext_suggestion = cleantext_suggestion[0]
    if not cleantext_solution :
        cleantext_solution = np.nan
    else :
        cleantext_solution = cleantext_solution[0]
    if not cleantext_vote :
        cleantext_vote = np.nan
    else :
        cleantext_vote = cleantext_vote[0]
    if not cleantext_reference :
        cleantext_reference = np.nan
    else :
        cleantext_reference = cleantext_reference[0]
            
    # 데이터프레임에 추가
    new_row = {'분류' : etc
               ,'프로젝트 종류' : status
               ,'프로젝트 시작' : start
               ,'제목' : title
               ,'미추진 사유' : cleantext_reason
               ,'상세 내용' : cleantext_content
               ,'제안의시작' : cleantext_cuase
               ,'주요제안(문제정의)' : cleantext_suggestion
               ,'해결방안' : cleantext_solution
               ,'공감투표 참여안내' : cleantext_vote
               ,'참고자료' : cleantext_reference
              }
    df = df.append(new_row, ignore_index=True)
    
    d.back()
    
    sleep(1)

df

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.97)
Stacktrace:
Backtrace:
	(No symbol) [0x00CA6643]
	(No symbol) [0x00C3BE21]
	(No symbol) [0x00B3DA9D]
	(No symbol) [0x00B1EF6A]
	(No symbol) [0x00B93AAB]
	(No symbol) [0x00BA61B6]
	(No symbol) [0x00B8FB76]
	(No symbol) [0x00B649C1]
	(No symbol) [0x00B65E5D]
	GetHandleVerifier [0x00F1A142+2497106]
	GetHandleVerifier [0x00F485D3+2686691]
	GetHandleVerifier [0x00F4BB9C+2700460]
	GetHandleVerifier [0x00D53B10+635936]
	(No symbol) [0x00C44A1F]
	(No symbol) [0x00C4A418]
	(No symbol) [0x00C4A505]
	(No symbol) [0x00C5508B]
	BaseThreadInitThunk [0x765A00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77C77BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77C77B8E+238]


In [6]:
df.to_csv("df.csv"
         , encoding="utf-8-sig"
         )

# # 다른 분류 데이터 모두 수집

In [31]:
# 가치더함 진입
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
address_jejudsi="https://jejudsi.kr/issue.htm"
d.get(address_jejudsi)
req = requests.get(address_jejudsi,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

# 새로운 데이터프레임 생성
col = ['id 값'
       ,'분류'
       ,'프로젝트 종류'
       ,'프로젝트 시작'
       ,'제목'
       ,'미추진 사유'
       ,'상세 내용'
       ,'제안의시작'
       ,'주요제안(문제정의)'
       ,'해결방안'
       ,'공감투표 참여안내'
       ,'참고자료'
       ,'링크'
      ]
df = pd.DataFrame(columns=col) 

for i in range(2,9+1) :
    xpath_tmp = f'/html/body/div[1]/section/div/div/div[2]/div/div/div/div/nav/div[2]/ul/li[{i}]/a'
    d.find_element(By.XPATH,xpath_tmp).click()
    
    # 카테고리 추출
    category = d.find_element(By.XPATH,xpath_tmp).text  
    '''
    1 = 전체보기
    2 = 보건복지
    3 = 산업경제
    4 = 도시교통
    5 = 문화교육
    6 = 관광
    7 = 환경
    8 = 농축수산
    9 = 기타
    '''
    
    # 더보기 무한 클릭
    path_more ='/html/body/div[1]/section/div/div/div[2]/div/div/div/div/div[7]/div[2]/a'
    while True :
        try :
            d.find_element(By.XPATH,path_more).click()
        except :
            break   

    # 프로젝트 진입 주소 수집
    html_2 = d.page_source
    soup_2 = BeautifulSoup(html_2, "html.parser")
    list_issueTit = soup_2.find_all('div', attrs={"class" : "issue-tit"})

    list_tag_issueTit = list()

    for i_list_issueTit in list_issueTit :
        list_tag_issueTit.append(i_list_issueTit.a['href'])

    for i_list_tag_issueTit in list_tag_issueTit :
        # 프로젝트 진입
        d.get(f'https://jejudsi.kr{i_list_tag_issueTit}')
        
        path_link = f'https://jejudsi.kr{i_list_tag_issueTit}'

        # 프로젝트 종류
        path_status ='/html/body/div[1]/section/div[1]/div/p[1]/span'
        status = d.find_element(By.XPATH, path_status).text    

        # 프로젝트 시작 시점
        try :        
            path_start = '/html/body/div[1]/section/div[2]/div/div[3]/div[1]/div[1]/div/div[2]/h3'
            start = d.find_element(By.XPATH, path_start).text
            start = start[9:19]
        except :
            start = np.nan

        # 프로젝트 제목
        title = d.find_element(By.CLASS_NAME, 'text-center').text

        # 내용 수집
        html_test = d.page_source
        soup_test = BeautifulSoup(html_test, "html.parser")

        # mt-3 섹션
        list_test = soup_test.find_all('div', attrs={"class" : "card mt-3"})

        list_list_test = list()
        for i_list_test in list_test :
            cleantext = i_list_test.text
            i_list_test = re.sub("\n", ".", cleantext)
            list_list_test.append(i_list_test)
        list_list_test

        cleantext_reason = list()
        cleantext_content = list()
        cleantext_cuase = list()
        cleantext_suggestion = list()
        cleantext_solution = list()
        cleantext_vote = list()
        cleantext_reference = list()

        for i_list_list_test in list_list_test :        
            if '미추진 사유' in i_list_list_test[:15] :
                cleantext_reason.append(i_list_list_test)
            elif '상세 내용' in i_list_list_test[:15] :
                cleantext_content.append(i_list_list_test)
            elif '제안의 시작' in i_list_list_test[:15] :
                cleantext_cuase.append(i_list_list_test) 
            elif '주요제안' in i_list_list_test[:15] :
                cleantext_suggestion.append(i_list_list_test)
            elif '해결방안' in i_list_list_test[:15] :
                cleantext_solution.append(i_list_list_test)          
            elif '공감투표' in i_list_list_test[:15] :
                cleantext_vote.append(i_list_list_test)
            elif '관련 자료' in i_list_list_test[:15] :
                cleantext_reference.append(i_list_list_test)
            elif '댓글' in i_list_list_test[:15] :
                pass
            else :
                pass

        if not (cleantext_reason and \
              cleantext_content and \
              cleantext_cuase and \
              cleantext_suggestion and \
              cleantext_solution and \
              cleantext_vote and \
              cleantext_reference) :

            # mt-t-10 섹션
            list_test = soup_test.find_all('div', attrs={"class" : "card m-t-10"})

            list_list_test = list()
            for i_list_test in list_test :
                cleantext = i_list_test.text
                i_list_test = re.sub("\n", ".", cleantext)
                list_list_test.append(i_list_test)

            for i_list_list_test in list_list_test :        
                if '미추진 사유' in i_list_list_test[:15] :
                    cleantext_reason.append(i_list_list_test)
                elif '상세 내용' in i_list_list_test[:15] :
                    cleantext_content.append(i_list_list_test)
                elif '제안의 시작' in i_list_list_test[:15] :
                    cleantext_cuase.append(i_list_list_test) 
                elif '주요제안' in i_list_list_test[:15] :
                    cleantext_suggestion.append(i_list_list_test)
                elif '해결방안' in i_list_list_test[:15] :
                    cleantext_solution.append(i_list_list_test)          
                elif '공감투표' in i_list_list_test[:15] :
                    cleantext_vote.append(i_list_list_test)
                elif '관련 자료' in i_list_list_test[:15] :
                    cleantext_reference.append(i_list_list_test)
                elif '댓글' in i_list_list_test[:15] :
                    pass
                else :
                    pass
        else :
            pass

        if not cleantext_reason :
            cleantext_reason = np.nan
        else :
            cleantext_reason = cleantext_reason[0]
        if not cleantext_content :
            cleantext_content = np.nan
        else :
            cleantext_content = cleantext_content[0]
        if not cleantext_cuase :
            cleantext_cuase = np.nan
        else :
            cleantext_cuase = cleantext_cuase[0]
        if not cleantext_suggestion :
            cleantext_suggestion = np.nan
        else :
            cleantext_suggestion = cleantext_suggestion[0]
        if not cleantext_solution :
            cleantext_solution = np.nan
        else :
            cleantext_solution = cleantext_solution[0]
        if not cleantext_vote :
            cleantext_vote = np.nan
        else :
            cleantext_vote = cleantext_vote[0]
        if not cleantext_reference :
            cleantext_reference = np.nan
        else :
            cleantext_reference = cleantext_reference[0]

        # 데이터프레임에 추가
        new_row = {'id 값' : id
                   ,'분류' : category
                   ,'프로젝트 종류' : status
                   ,'프로젝트 시작' : start
                   ,'제목' : title
                   ,'미추진 사유' : cleantext_reason
                   ,'상세 내용' : cleantext_content
                   ,'제안의시작' : cleantext_cuase
                   ,'주요제안(문제정의)' : cleantext_suggestion
                   ,'해결방안' : cleantext_solution
                   ,'공감투표 참여안내' : cleantext_vote
                   ,'참고자료' : cleantext_reference
                   ,'링크' : path_link
                  }
        df = df.append(new_row, ignore_index=True)

        d.back()

        sleep(1)

df        

,분류,프로젝트 종류,프로젝트 시작,제목,미추진 사유,상세 내용,제안의시작,주요제안(문제정의),해결방안,공감투표 참여안내,참고자료,링크
0,보건・복지,프로젝트완료,NaN,무장애 여행을 위한 지도 및 데이터 구축,NaN,".. 상세 내용...개요..장애인여행의 가치는 단순히 여가를 넘어 인간다운 삶, 즉...",NaN,NaN,NaN,NaN,.. 관련 자료 ... 다운로드.\t\t\t\t\t\t관광약자를 위한 제주 여행지도...,https://jejudsi.kr/issue/RpGP
1,보건・복지,기간종료,2022-08-10,장애인 지원,NaN,NaN,.. 제안의 시작 (문제정의).제가 6년전에 뇌경색으로 쓰러져 편마비가 와서 외부활...,NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/A8vM
2,보건・복지,기간종료,2022-04-21,동네 병원 정보와 진료 정보를 한번에,NaN,NaN,.. 제안의 시작 (문제정의).\t\t\t\t정기적으로 병원을 다니면서 약을 처방받...,NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/A9ne
3,보건・복지,기간종료,2021-12-20,제주안심앱을 이용한 백신접종 확인,NaN,NaN,.. 제안의 시작 (문제정의).\t\t\t\t12월18일부터 코로나 방역4단계 진입...,NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/L4nP
4,보건・복지,기간종료,2021-12-16,안전조끼(용품) 착용 캠페인,NaN,NaN,".. 제안의 시작 (문제정의).\t\t\t\t제주도는 빛공해 방지를 위한 조도저감,...",NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/LjYE
...,...,...,...,...,...,...,...,...,...,...,...,...
150,기타,기간종료,2020-09-21,안전귀가,NaN,NaN,".. 제안의 시작 (문제정의).- 제안 취지(제안에 관심을 가지게 된 이유, 계기)...",NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/LGxY
151,기타,기간종료,2020-07-07,전기차 보급에 대한 의견,NaN,NaN,".. 제안의 시작 (문제정의).- 제안 취지(제안에 관심을 가지게 된 이유, 계기)...",NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/17mn
152,기타,기간종료,2020-06-25,실종견 찾는 지역 어플,NaN,NaN,".. 제안의 시작 (문제정의).- 제안 취지(제안에 관심을 가지게 된 이유, 계기)...",NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/R3DE
153,기타,기간종료,2020-05-04,★제주도 지자체운영 공공배달앱 으로 ★재능기부 관련★ 상담 미팅 신청,NaN,NaN,".. 제안의 시작 (문제정의).- 제안 취지(제안에 관심을 가지게 된 이유, 계기)...",NaN,NaN,NaN,NaN,https://jejudsi.kr/issue/RwEr


In [33]:
df.to_csv("df_가치더함.csv"
          , encoding="utf-8-sig"
          , index=False
         )

In [6]:
# 가치더함 진입
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/chromedriver/chromedriver.exe') # webdriver = chrome
d.implicitly_wait(3)
address_jejudsi="https://jejudsi.kr/issue.htm"
d.get(address_jejudsi)
req = requests.get(address_jejudsi,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

In [ ]:
# 새로운 데이터프레임 생성
col = ['분류'
       ,'프로젝트 종류'
       ,'프로젝트 시작'
       ,'제목'
       ,'미추진 사유'
       ,'상세 내용'
       ,'제안의시작'
       ,'주요제안(문제정의)'
       ,'해결방안'
       ,'공감투표 참여안내'
       ,'참고자료'
       ,'링크'
      ]
df = pd.DataFrame(columns=col) 

In [8]:
xpath_tmp = f'/html/body/div[1]/section/div/div/div[2]/div/div/div/div/nav/div[2]/ul/li[{2}]/a'
d.find_element(By.XPATH,xpath_tmp).click()

In [9]:
category = d.find_element(By.XPATH,xpath_tmp).text  

In [10]:
category

'보건・복지'

In [12]:
path_more ='/html/body/div[1]/section/div/div/div[2]/div/div/div/div/div[7]/div[2]/a'
while True :
    try :
        d.find_element(By.XPATH,path_more).click()
    except :
        break

In [13]:
# 프로젝트 진입 주소 수집
html_2 = d.page_source
soup_2 = BeautifulSoup(html_2, "html.parser")
list_issueTit = soup_2.find_all('div', attrs={"class" : "issue-tit"})

In [19]:
list_issueTit

[<div class="issue-tit">
 <a href="/issue/RpGP">시민참여를 통한 무장애여행 데이터 수집/개방</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/A8vM">장애인 지원</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/A9ne">동네 병원 정보와 진료 정보를 한번에</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/L4nP">제주안심앱을 이용한 백신접종 확인</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/LjYE">안전조끼(용품) 착용 캠페인</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/1Eyy">길 잃은 치매 노인을 위한 디지털 신상정보 관리 시스템</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/1zD0">독거노인을 위한 전용 메신저(대행플랫폼)</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/Lj9K">장애인의 건강</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/1dZK">두박스(Donation Box) : 기부자 – 저소득층 기부물품 직접 연결 플랫폼</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/RvMO">비대면 체온측정, 얼굴인식, 바이러스 소독까지 가능한 시스템 개발 및 비치</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/1VxX">이동약자 맞춤 콜 서비스</a>
 </div>,
 <div class="issue-tit">
 <a href="/issue/AmbZ">제 2공항 연기에 따른 주민 피해와 단기적 제안</a

In [15]:
list_tag_issueTit = list()

In [17]:
for i_list_issueTit in list_issueTit :
        list_tag_issueTit.append(i_list_issueTit.a['href'])

In [24]:
d.get(f'https://jejudsi.kr'+list_tag_issueTit[0])

In [26]:
path_link = f'https://jejudsi.kr' + list_tag_issueTit[0]

In [27]:
path_status ='/html/body/div[1]/section/div[1]/div/p[1]/span'
status = d.find_element(By.XPATH, path_status).text

In [28]:
status

'프로젝트완료'

In [ ]:
try :        
    path_start = '/html/body/div[1]/section/div[2]/div/div[3]/div[1]/div[1]/div/div[2]/h3'
    start = d.find_element(By.XPATH, path_start).text
    start = start[9:19]
except :
    start = np.nan